In [1]:
# install libraries
!pip install langchain
!pip install langgraph
!pip install langchain-openai
!pip install langchain-core

In [2]:
# Import all the libraries required 
from flask import Flask, request, jsonify
from threading import Thread
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from langchain.tools import tool
from datetime import datetime
import time
from langchain_openai import ChatOpenAI
from langgraph.graph import START, END
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field 
from langchain.output_parsers import PydanticOutputParser



In [4]:
## Configure OpenAI API for vLLM Model
base_url = "http://localhost:3000/v1"
api_key = "goutham"
headers = {"Content-Type": "application/json"}
#llm = ChatOpenAI(model= "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat",base_url=base_url,api_key=api_key,default_headers=headers)
llm = ChatOpenAI(model= "Qwen/Qwen2.5-14B-Instruct",base_url=base_url,api_key=api_key,default_headers=headers)

In [6]:
## Define all the Tools and Functions required
import os

#BASE_DIR = os.path.dirname(os.path.abspath(__file__))
#token_path = os.path.join(BASE_DIR, "Keys", user.split("@")[0] + ".token")

@tool
def retrive_calendar_events(users: str,start: str, end:str ):
    """Fetch calendar events for `users` between ISO‑8601 `start` and `end`."""
    # parameters = json.loads(parameters)
    # user = parameters.get("user")
    # start = parameters.get("start")
    # end = parameters.get("end")
    user_events = []
    for user in users:
        events_list = []
        token_path = "Keys/"+user.split("@")[0]+".token"
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
        events = events_result.get('items')
        
        for event in events : 
            attendee_list = []
            try:
                for attendee in event["attendees"]: 
                    attendee_list.append(attendee['email'])
            except: 
                attendee_list.append("SELF")
            try:
                start_time = event["start"]["dateTime"]
                end_time = event["end"]["dateTime"]
            except KeyError:
                start_time = event["start"]
                end_time = event["end"]
            events_list.append(
                {"StartTime" : start_time, 
                 "EndTime": end_time, 
                 "NumAttendees" :len(set(attendee_list)), 
                 "Attendees" : list(set(attendee_list)),
                 "Summary" : event["summary"]})
        if len(events_list)==0:
            events_list.append(["User Doesn't have any meetings scheduled for this time slot"])
        #return events_list
        user_events.append({user:events_list})
    return user_events
def retrive_calendar_events_not_tool_call(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        try:
            start_time = event["start"]["dateTime"]
            end_time = event["end"]["dateTime"]
        except KeyError:
            start_time = event["start"]
            end_time = event["end"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [7]:
from datetime import datetime
import pytz

def convert_date_to_ist_datetime(date_string):
    """
    Convert date string to IST datetime with timezone
    """
    # Parse the date string
    date_obj = datetime.strptime(date_string, "%Y-%m-%d")
    
    # Set timezone to IST
    ist_timezone = pytz.timezone('Asia/Kolkata')
    
    # Localize the datetime to IST
    ist_datetime = ist_timezone.localize(date_obj)
    
    # Format to desired output
    output = ist_datetime.strftime("%Y-%m-%dT%H:%M:%S%z")
    
    return output

In [8]:
# Structure for Agent 
class Structured_Output_llm(BaseModel):
    start_time: str = Field(description="Proposed Start Time")
    end_time: str = Field(description="Proposed End Time")
    duration: int = Field(description="duration between Start time and End time")
    optimal_start_time: str = Field(description=" Conflict free start date/time if there is a conflict with others on proposed date")
    optimal_end_time: str = Field(description= "Conflict free end date/time if there is a conflict with others on proposed date")
    #is_there_a_conflict: bool = Field(description= "Is there a conflict in proposed date and time?")
parser = PydanticOutputParser(pydantic_object=Structured_Output_llm)



def load_json(file_path: str):
    """Read a JSON file and return the parsed Python object."""
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

In [9]:
app = Flask(__name__)
received_data = []

In [ ]:
# def your_meeting_assistant(data): 
#     # Your Agentic AI Calls 
#     data["EventStart"] = ""
#     data["EventEnd"] = ""
#     data["Duration_mins"] = ""
#     return data

In [15]:
def agent_function(input_data):
    st_time = time.time()
    from_mail = input_data.get("From")
    email_content = input_data.get("EmailContent")
    email_prompt = f"""
    
    You are an expert meeting scheduler. Your task is to schedule meetings by analyzing user requests and calendar availability.
    
    ## CRITICAL CONSTRAINTS
    - **ONLY VALID USERS**: userone.amd@gmail.com, usertwo.amd@gmail.com, userthree.amd@gmail.com
    - **REFERENCE DATE**: 2025-07-20 (Sunday)
    - **TIMEZONE**: IST (+05:30)
    - **WORKING DAYS**: weekdays only
    
    ## MEETING CLASSIFICATION (EXACT MATCHING)
    
    ### P0 - OUTAGE MEETING (CRITICAL)
    **Keywords**: emergency, urgent, ASAP, outage, crash, failure, breach, crisis, critical
    **Rules**: 
    - Override ALL conflicts (leaves, meetings, weekends, off-hours)
    - Schedule immediately (within 30 minutes if possible)
    - All invitees must attend
    
    ### P1 - CLIENT/CUSTOMER MEETING (HIGH)
    **Keywords**: client, customer, demo, feedback, requirements, quarterly review
    **Rules**:
    - Override internal meetings only
    - No weekends or off-hours
    - Reschedule if participants on leave/external events
    
    ### P2 - INTERNAL MEETING (STANDARD)
    **Keywords**: team, 1-on-1, sprint, project, catchup, planning, retrospective, brainstorming
    **Rules**:
    - Can be rescheduled for higher priority meetings
    - Not to schedule on weekends or off-hours
    - Reschedule if participants unavailable
    
    ## STEP-BY-STEP PROCESS
    
    ### Step 1: Extract Meeting Type
    Look for these EXACT keywords in the request:
    - **OUTAGE**: emergency, urgent, ASAP, outage, crash, failure, breach, crisis, critical
    - **CLIENT**: client, customer, demo, feedback, requirements, quarterly review  
    - **INTERNAL**: team, 1-on-1, sprint, project, catchup, planning, retrospective, brainstorming
    
    ### Step 2: Extract Date/Time
    - **Explicit date**: Use as-is
    - **Day name**: Convert using reference date (2025-07-20 = Sunday)
      - Monday = 2025-07-21, Tuesday = 2025-07-22, etc.
    - **Relative**: "tomorrow" = 2025-07-21, "next Monday" = 2025-07-28
    - **Default**: If no date specified, use next business day
    
    ### Step 3: Validate Constraints
    - **Past dates**: Reject if before 2025-07-20
    - **Weekends**: Only outage meetings allowed
    - **Off-hours**: Only outage and client meetings allowed
    
    ### Step 4: Check Availability
    1. Use calendar tool to fetch events for all participants
    2. Find earliest available slot on proposed date
    3. If conflict exists, check next business day
    4. Continue until conflict-free slot found
    
    ### Step 5: Select Time Slot
    - **Earliest available** during working hours (9 AM - 6 PM)
    - **All participants free**
    - **Respect priority rules**
    
    ## PRIORITY RESOLUTION MATRIX
    
    | Request Type | vs Outage | vs Client | vs Internal | vs Leave/External |
    |--------------|-----------|-----------|-------------|-------------------|
    | **Outage**   | Override  | Override  | Override    | Override          |
    | **Client**   | Defer     | Defer     | Override    | Defer             |
    | **Internal** | Defer     | Defer     | Defer       | Defer             |
    
    ## OUTPUT REQUIREMENTS
    - **ONLY OUTPUT THE JSON OBJECT - NO EXPLANATORY TEXT**
    - **No markdown formatting**
    - **No additional text before or after the JSON**
    - **ISO 8601 format**: YYYY-MM-DDTHH:MM:SS+05:30
    - **Exact schema compliance**: {parser.get_format_instructions()}
    
    **CRITICAL**: Your response must contain ONLY the JSON object. Do not include any explanatory text, reasoning, or additional content.
    
    ## VALIDATION CHECKLIST
    Before outputting, verify:
    - [ ] All participants are valid users
    - [ ] Date is not in the past
    - [ ] Time is within working hours (unless outage)
    - [ ] No conflicts with higher priority meetings
    - [ ] All participants are available
    - [ ] JSON format is correct
    
    ## EXAMPLE PATTERNS
    - "emergency call" → OUTAGE (P0)
    - "client meeting" → CLIENT (P1)  
    - "team meeting" → INTERNAL (P2)
    - "tomorrow" → 2025-07-21
    - "next Monday" → 2025-07-28
    
    **IMPORTANT**: If uncertain about any aspect, choose the most conservative option (lower priority, later time, next business day).
    """
    
    
    llm_tools = llm.bind_tools([retrive_calendar_events])
    agent = create_react_agent(model = llm_tools, tools =[retrive_calendar_events])
    
    result = agent.invoke({"messages":[{"role":"system","content":email_prompt},{"role":"user","content":f"The mail from {from_mail} with emai_content is {email_content}"}]})
    assistant_msg = result["messages"][-1].content          # latest assistant reply
    structured    = parser.parse(assistant_msg)             # → SchedulerResponse object
    print(structured.model_dump())  # or whatever downstream logic you need
    
    ## Write data to Json format
    
    output_data = input_data.copy() 
    
    output_data["EventStart"] = structured.optimal_start_time  
    output_data["EventEnd"] = structured.optimal_end_time 
    output_data["Duration_mins"] = str(structured.duration)
    output_data["MetaData"] = []
    del output_data["Attendees"]
    # Get list of Events of Attendees - You also need to implement for Scenario's where you need to replace the meeting later 
    list_of_users = [input_data.get("From")] + [m.get("email") for m in input_data.get("Attendees")]
    
    start_time = structured.start_time
    end_time = output_data["EventEnd"]
    output_data["Attendees"] = []
    
    st_dt = datetime.fromisoformat(start_time)
    start_of_day = st_dt.replace(hour=9, minute=0, second=0, microsecond=0).isoformat()
    
    en_dt = datetime.fromisoformat(end_time)
    end_of_day = en_dt.replace(hour=23, minute=59, second=0, microsecond=0).isoformat()
    #print("start_of_day", start_of_day)
    #print("end_of_day", end_of_day)
    
    ## Write code for range of dates - proposed to arranged date 
    
    en_dt_optimal = datetime.fromisoformat(structured.optimal_end_time)
    end_of_day_optimal = en_dt_optimal.replace(hour=23, minute=59, second=0, microsecond=0).isoformat()
    
    ## Code - if there is any overlap event - postpone by +1 day 
    print(list_of_users)
    for user in list_of_users:
        proposed_meeting = [{"StartTime" : output_data["EventStart"], 
                 "EndTime": end_time, 
                 "NumAttendees" :len(set(list_of_users)), 
                 "Attendees" : list(set(list_of_users)),
                 "Summary" : input_data["Subject"]}]
        moved_events = []
        new_events = []
        events = retrive_calendar_events_not_tool_call(user, start_of_day, end_of_day_optimal)
        if(len(events)>0):
            for event in events:
                if isinstance(event["StartTime"], dict):
                    event_start_date = convert_date_to_ist_datetime(event["StartTime"]["date"])
                    event_end_date = convert_date_to_ist_datetime(event["EndTime"]["date"])
                else:
                    event_start_date = event["StartTime"]
                    event_end_date = event["EndTime"]
                event_start = datetime.fromisoformat(event_start_date)
                event_end = datetime.fromisoformat(event_end_date)
                opt_start = datetime.fromisoformat(output_data["EventStart"])
                opt_end = datetime.fromisoformat(output_data["EventEnd"])
                
    
                
                if event_start < opt_end and opt_start < event_end:
                    new_start = opt_end + timedelta(hours=2)
                    new_end = new_start + (event_end - event_start)
                    #event["StartTime"] = new_start
                    #event["EndTime"] = new_end
                    new_start = new_start.isoformat()
                    new_end = new_end.isoformat()
                    moved_events.append({"StartTime": new_start,'EndTime': new_end,'NumAttendees': event["NumAttendees"],'Attendees': event["Attendees"],'Summary': event["Summary"]})
                else:
                    new_events.append(event)
                
        attendee_record = {
            "email": user,
            #"events": retrive_calendar_events_not_tool_call(user, start_of_day, end_of_day_optimal) + proposed_meeting 
            "events": new_events + proposed_meeting + moved_events
        }
        output_data["Attendees"].append(attendee_record)    
    input_date = datetime.strptime(output_data["Datetime"], "%d-%m-%YT%H:%M:%S")    
    output_data["Datetime"] = input_date.strftime("%Y-%m-%dT%H:%M:%S")
    et_time = time.time()
    duration = et_time - st_time 
    output_data["MetaData"] = [{"latency":duration}]
    return output_data

In [11]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    #print(f"\n Received: {json.dumps(data, indent=2)}")
    #new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    #received_data.append(data)
    #print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    new_data = agent_function(data)
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [12]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.191.94:5000
Press CTRL+C to quit


{'start_time': '2025-07-24T10:00:00+05:30', 'end_time': '2025-07-24T10:30:00+05:30', 'duration': 30, 'optimal_start_time': '2025-07-24T10:00:00+05:30', 'optimal_end_time': '2025-07-24T10:30:00+05:30'}


14.139.128.62 - - [20/Jul/2025 08:09:25] "POST /receive HTTP/1.1" 200 -


{'start_time': '2025-07-24T14:00:00+05:30', 'end_time': '2025-07-24T14:30:00+05:30', 'duration': 30, 'optimal_start_time': '2025-07-24T14:00:00+05:30', 'optimal_end_time': '2025-07-24T14:30:00+05:30'}


14.139.128.62 - - [20/Jul/2025 08:14:50] "POST /receive HTTP/1.1" 200 -


{'start_time': '2025-07-24T14:00:00+05:30', 'end_time': '2025-07-24T14:30:00+05:30', 'duration': 30, 'optimal_start_time': '2025-07-24T14:00:00+05:30', 'optimal_end_time': '2025-07-24T14:30:00+05:30'}
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']


14.139.128.62 - - [20/Jul/2025 08:15:24] "POST /receive HTTP/1.1" 200 -


{'start_time': '2025-07-24T10:00:00+05:30', 'end_time': '2025-07-24T10:30:00+05:30', 'duration': 30, 'optimal_start_time': '2025-07-24T10:00:00+05:30', 'optimal_end_time': '2025-07-24T10:30:00+05:30'}
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']


14.139.128.62 - - [20/Jul/2025 08:18:47] "POST /receive HTTP/1.1" 200 -


In [13]:
print(os.getcwd())

/home/user/AI-Scheduling-Assistant
